## A Guide: Turning OpenStreetMap Location Data into ML Features
#### How to pull shops, restaurants, public transport modes and other local amenities into your ML models.

#### 1. Import libraries and load data from database.

Import Python libraries

Load dataset from database with read_sql_table

Define feature and target variables X and Y

In [ ]:
pip install osmnx

In [ ]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
import os
import pickle
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path

# import modules for visualisation
import seaborn as sns
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt 

pd.set_option('display.max_columns', None)

from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

In [ ]:
cwd = Path().resolve()
dirname = os.path.join(Path(cwd).parent, 'data', 'listings.csv')

# import the airbnb data
df = pd.read_csv(dirname, index_col=False, sep=",")
df_cal = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'calendar.csv'), index_col=False, sep=",")
df_rev = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'reviews.csv'), index_col=False, sep=",")

# set data types
pd.to_datetime(df['last_review'])
pd.to_datetime(df_cal['date'])
pd.to_datetime(df_rev['date'])

# display the first rows...
df.head(1)

In [ ]:
# manipulate data
def get_price(price_str):
    price = price_str.replace('$','')
    return price.replace('.', '')
df_cal['price'] = df_cal.apply(lambda x: get_price(x['price']), axis=1)
df_cal['price'] = df_cal['price'].astype(float)

In [ ]:
df_cal.head(5)

In [ ]:


# plot figure with 4 line charts showing availability and price deviation throughout the year 
fig = plt.figure(figsize=(16, 8), constrained_layout = False)

# setup grid layout
outer_grid = fig.add_gridspec(1, 2, wspace = 0.02, hspace = 0)
for i in range(2):
    inner_grid = outer_grid[i].subgridspec(2, 1, wspace = 0, hspace = 0.05)
    for j in range(2):
        fig.add_subplot(inner_grid[j])

ax = fig.get_axes()

# plot for each price category
for idx, price_bin in enumerate(price_category):
    
    # graph index shortcuts (ax_av = avilability graph, ax_price = price graph)
    ax_av = ax[2 * idx]
    ax_price = ax[2 * idx + 1]
    

    # prepare subset of data
    data = trend_all[(trend_all['price_category'] == price_bin)]
        
    # plot daily availability of listings
    data.plot(x='date', y='available_daily_subgroup_avg', legend = False, linewidth = 2, ax = ax_av)
        
    # plot price percent deviation from mean daily prices
    data.plot(x='date', y='adjusted_price_daily_pdev', legend = False, linewidth = 2, ax = ax_price)    
    
    # formatting of axes, legends, titles
    ax_av.set_title(f'price category: {price_bin.capitalize()}')
    ax_av.axes.get_xaxis().set_visible(False)
    ax_av.set_ylim(0, 1)
    ax_price.set_xlabel('')
    ax_price.set_ylim(-35, 100)
    
    # add dashed line to mark 0 % for price percent deviation charts
    ax_price.axhline(y = 0, color = 'k', linestyle = 'dashed', linewidth=1)


# clean up ticks and labels
ax[0].set_ylabel('proportion of available listings\n(0 - 1)', fontsize=12)
ax[1].set_ylabel('price deviation in percent\nfrom average (%)', fontsize=12)
ax[2].set_yticks([])
ax[3].set_yticks([])
fig.suptitle(f'Airbnb listing availability and price trends in course of the year'
               f'\nfrom {data.date.min().date()} ' f'to {data.date.max().date()}', y=1.00)

You could loop this process and also pull all data on bars, restaurants, nightclubs, and all of the other amenities that might affect what a host will charge for an Airbnb.

In [ ]:
query = {'amenity':'restaurant'}

restaurants_gdf = ox.pois.pois_from_place(
            'Greater London, UK',
            tags = query,
            which_result=1)

restaurants_gdf.head(5)

In [ ]:
def get_geo_data(selector, sel='offline'):
    """ Load Austrian geojson  """

    if selector == 'municipal' and sel=='online':
        link = 'https://raw.githubusercontent.com/ginseng666/GeoJSON-TopoJSON-Austria/master/2021/simplified-99.5/gemeinden_995_geo.json'
    elif selector == 'municipal' and sel=='offline':
        with open(os.path.join(Path(cwd).parent, 'data', 'geojson', 'gemeinden_999_geo.json')) as fp:
            counties = json.load(fp)

    if selector == 'district' and sel=='online':
            link = 'https://raw.githubusercontent.com/ginseng666/GeoJSON-TopoJSON-Austria/master/2021/simplified-99.9/bezirke_999_geo.json'
    elif selector == 'district' and sel=='offline':
        with open(os.path.join(Path(cwd).parent, 'data', 'geojson', 'bezirke_999_geo.json')) as fp:
            counties = json.load(fp)
        
    if selector == 'state' and sel=='online':
        link = 'https://raw.githubusercontent.com/ginseng666/GeoJSON-TopoJSON-Austria/master/2021/simplified-99.9/laender_999_geo.json'
    elif selector == 'state' and sel=='offline':
        with open(os.path.join(Path(cwd).parent, 'data', 'geojson', 'laender_999_geo.json')) as fp:
            counties = json.load(fp)
            
    if sel == 'online':
        with urlopen(link) as response:
            counties = json.load(response)

    return counties

In [ ]:
# explore data
### display histograms for all columns with numerical data
listings_df.select_dtypes(include=np.number).hist(figsize = (16, 10), bins=50, log = False)
plt.suptitle('distributions of numeric features', y=1.05)
plt.tight_layout()


In [ ]:
path = os.path.join(Path(cwd).parent, 'data', 'neighbourhoods.geojson')
neighbourhoods_gdf = gpd.read_file(path)
display(neighbourhoods_gdf.plot())

#### 2. OSM Data
##### 2.1. Query OSM Data
source: https://python.plainenglish.io/osmnx-the-fastest-way-to-get-data-from-openstreetmaps-731419d4dc31

Overpass frontend: https://overpass-turbo.eu/

Overpass doku: https://wiki.openstreetmap.org/wiki/Overpass_API

Countries are marked (2), states (4), cities can (6) or (7) depending on type 

https://wiki.openstreetmap.org/wiki/Overpass_API/Language_Guide#Select_Region_by_Polygon


Cool osmnx example: https://python.plainenglish.io/osmnx-the-fastest-way-to-get-data-from-openstreetmaps-731419d4dc31

osmnx doku:  https://osmnx.readthedocs.io/en/stable/osmnx.html

insight airbnbn vienna: https://github.com/eugeniftimoaie/airbnb_vienna/blob/master/airbnb_vienna.ipynb

https://towardsdatascience.com/a-guide-turning-openstreetmap-location-data-into-ml-features-e687b66db210

https://www.data.gv.at/katalog/dataset/stadt-wien_bezirksgrenzenwien


https://gis.stackexchange.com/questions/354184/showing-only-the-external-boundary-of-a-geopandas-dataframe




In [195]:
region_name = 'Vienna Inner City, Austria'#
#
# https://nominatim.openstreetmap.org/reverse?format=xml&lat=17.3616&lon=78.4747&zoom=18&addressdetails=1
#or 'City, Country' if you have city that meets more ...than one time in the world
#sometimes you need to play with which_result parameter because you can receive node point, not the polygon



#region = ox.geocoder.geocode_to_gdf(region_name, which_result=2, by_osmid=False)
region.plot(figsize=(15,15))
plt.title(region_name, fontdict={'fontsize':15})
plt.grid()

AttributeError: 'Polygon' object has no attribute 'plot'

In [196]:
boundary_geojson = gpd.read_file(os.path.join(Path(cwd).parent, 'data', 'geojson', 'vienna.geojson'))
region = boundary_geojson.geometry.unary_union

C:\Users\klara\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [200]:
#print(region)

In [ ]:
# 'N17328659'

polygon = ox.geocode_to_gdf(['R109166'])
#polygon = gdf.iloc[0]['geometry']
#polygon.exterior.coords
polygon.plot(figsize=(15,15))
plt.grid()

In [ ]:
gdf = ox.geocode_to_gdf('N17328659', by_osmid=True)
polygon = gdf.iloc[0]['geometry']
polygon.exterior.coords
list(polygon.exterior.coords)

#### 2.2. Get list of all the keys from the OSM Wiki
lookup: https://github.com/gboeing/osmnx/blob/main/tests/test_osmnx.py

In [ ]:
def get_all_amenities():
    """ get all amenitiy keys from OSM wiki """
    try:
        amenities = pd.read_html('https://wiki.openstreetmap.org/wiki/Key:amenity', skiprows = 0, header=0, attrs = {'class': 'wikitable'})[0]
        amenities.drop(columns=['Element', 'Carto rendering','Photo', 'Unnamed: 6'], inplace=True, axis=1)
        amenities.drop(index=0, inplace=True)
        return amenities
    except:
        print("Amenities could not be found")

        
def get_buildings():
    """ get all building keys from OSM wiki """
    try:
        building = pd.read_html('https://wiki.openstreetmap.org/wiki/Key:building', skiprows = 0, header=0, attrs = {'class': 'wikitable'})[0]
        building.drop(columns=['Unnamed: 4', 'Unnamed: 5', 'Photo', 'Unnamed: 6'], inplace=True, axis=1)
        building.drop(index=0, inplace=True)
        return building
    except:
        print("buildings could not be found")
        
buildings = get_buildings()
amenities = get_all_amenities()
print(f"Building keys: {', '.join(buildings['Value'].tolist())}")
print(f"Amenities keys: {', '.join(amenities['Value'].tolist())}")

#### 2.3. Query the date with the ox API

In [ ]:
t0 = time.time()

#building: True means that every type of buildings will be downloaded
buildings = ox.geometries.geometries_from_polygon(region['geometry'][0], tags = {'building': True})
#Using graph module we will recieve graph of roads in the city
roads = ox.graph.graph_from_polygon(region['geometry'][0])
restaurants = ox.geometries.geometries_from_polygon(region['geometry'][0], tags = {'amenity':'restaurant'})
kindergarten = ox.geometries.geometries_from_polygon(region['geometry'][0], tags = {'amenity':'kindergarten'})
cafe = ox.geometries.geometries_from_polygon(region['geometry'][0], tags = {'amenity':'cafe'})
bars = ox.geometries.geometries_from_polygon(region['geometry'][0], tags = {'amenity':'bar'})
forest = ox.geometries.geometries_from_polygon(region['geometry'][0], tags = {'landuse': 'forest'})
rivers = ox.geometries.geometries_from_polygon(region['geometry'][0], tags = {'waterway': 'river'})

print ("Completed in %s seconds" % (round(time.time() - t0, 2)))

In [ ]:
secondary_roads = ox.geometries.geometries_from_polygon(region['geometry'][0], tags = {'highway': 'secondary'})

In [ ]:
secondary_roads.head()

In [ ]:
def convert_xy(xy):
    """ """
    try:
        return pd.Series([xy.x, xy.y])
    except:
        print(xy)
        pass
    #restaurants['lon'] = restaurants['geometry'].x
    #restaurants['lat'] = restaurants['geometry'].y
    # restaurants.head()
    
parameters = [roads, restaurants, kindergarten, cafe, bars, rivers]
for i in parameters:
    try:
        print(t.shape[0])
        #j = t['geometry'].representative_point()
        i['geometry'] = i['geometry'].apply(lambda x: x.centroid if type(x) == Polygon else (x.centroid if type(x) == MultiPolygon else x))
        i[['x', 'y']] = i.apply(lambda x: convert_xy(x['geometry']), axis=1)
        print(i.shape[0])
    except:
        pass



In [ ]:
#restaurants['lon'] = restaurants['geometry'].point_object.apply(lambda p: p.x)
#restaurants['lat'] = restaurants['geometry'].point_object.apply(lambda p: p.y)
i =restaurants.copy()
i.head()

In [ ]:
#city
ax = region.plot(facecolor = '#494D4D', figsize=(85,85))
ax.set_facecolor('#2C2E2E')
buildings['geometry'].plot(facecolor = '#C61313',
                           edgecolor = '#C61313',
                           linewidth = 3,
                           markersize = 1,
                           ax = ax)
"""
forest.plot(facecolor = '#494D4D',
            edgecolor = '#ADC3B8',
            linewidth = 2,
            linestyle = ':',
            hatch = 'x',
            ax = ax)

rivers.plot(edgecolor = '#67A0C3',
            linewidth = 6,
            linestyle = '-',
            ax = ax)

ox.plot_graph(roads,
              edge_color = 'white',
              node_color = 'white',
              edge_linewidth=2,
              node_size = 2,
              ax=ax)"""

ax.grid('on', which='major', axis='x', color = '#99A2A2')
ax.grid('on', which='major', axis='y', color = '#99A2A2')
plt.show()

In [ ]:
def geo_heatmap(df, party, selector, camp, show_winner=True, title=''):
    """ selector specifies the geographic resolution
    """
    feat_key = ''
    locations = ''
    hover_data = ''
    if selector == 'municipal':
        feat_key = "properties.iso"
        locations = "GKZ"
        hover_data = party
        hover_name = df.NAME
    elif selector == 'district':
        feat_key = "properties.iso"
        locations = "BEZ"
        hover_data = party
        hover_name = df.NAME
    elif selector == 'state':
        feat_key = "properties.iso"
        locations = "LAND"
        hover_data = party
        hover_name = df.NAME
    temp = get_geo_data(selector, tpye='offline')

    fig = px.choropleth_mapbox(df, geojson=temp, locations=locations,
                               featureidkey=feat_key,
                               title=title,
                               
                               #color_continuous_scale="Viridis",
                               #range_color=(df[camp].quantile(0.25), df[camp].quantile(0.75)),
                               
                               # mapbox_style="carto-positron",
                               color_discrete_map= {'ÖVP': 'black',
                                      'SPÖ': 'red',
                                      'FPÖ': 'blue',
                                        'GRÜNE': 'green'},
                               color=df['winner'],
                               hover_data = hover_data,
                               hover_name = hover_name,
                               mapbox_style="open-street-map",
                               zoom=5, center = {"lat": 47.809490, "lon": 13.055010},
                               opacity=0.25,
                              )
    # fig.update_layout(margin={"r":10,"t":10,"l":0,"b":0})
    fig.update_layout(coloraxis_colorbar_title='Gewinner')
    fig.show()

In [ ]:
# download notebook to html
!jupyter nbconvert --to html airbnb_analysis.ipynb

# pip install --upgrade ipykernel

In [ ]:
# https://resonance-analytics.com/blog/deploying-dash-apps-on-azure
# ML engineer / data story teller / electrical engineer / digital illustrator / 3D modeler. I am writing data science blog with my cousin.


# wiener :
https://nbviewer.org/github/eugeniftimoaie/airbnb_vienna/blob/master/airbnb_vienna.ipynb
